In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict
from collections import Counter

pd.set_option('display.max_columns', 60)
pd.set_option('display.max_rows', 1000)

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Load the dataset for recommenders

In [2]:
data_path = os.path.join("data", "hotel_data")

interactions_df = pd.read_csv(os.path.join(data_path, "hotel_data_interactions_df.csv"), index_col=0)

base_item_features = ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']

column_values_dict = {
    'term': ['WinterVacation', 'Easter', 'OffSeason', 'HighSeason', 'LowSeason', 'MayLongWeekend', 'NewYear', 'Christmas'],
    'length_of_stay_bucket': ['[0-1]', '[2-3]', '[4-7]', '[8-inf]'],
    'rate_plan': ['Standard', 'Nonref'],
    'room_segment': ['[0-160]', '[160-260]', '[260-360]', '[360-500]', '[500-900]'],
    'n_people_bucket': ['[1-1]', '[2-2]', '[3-4]', '[5-inf]'],
    'weekend_stay': ['True', 'False']
}

interactions_df['term'] = pd.Categorical(
    interactions_df['term'], categories=column_values_dict['term'])
interactions_df['length_of_stay_bucket'] = pd.Categorical(
    interactions_df['length_of_stay_bucket'], categories=column_values_dict['length_of_stay_bucket'])
interactions_df['rate_plan'] = pd.Categorical(
    interactions_df['rate_plan'], categories=column_values_dict['rate_plan'])
interactions_df['room_segment'] = pd.Categorical(
    interactions_df['room_segment'], categories=column_values_dict['room_segment'])
interactions_df['n_people_bucket'] = pd.Categorical(
    interactions_df['n_people_bucket'], categories=column_values_dict['n_people_bucket'])
interactions_df['weekend_stay'] = interactions_df['weekend_stay'].astype('str')
interactions_df['weekend_stay'] = pd.Categorical(
    interactions_df['weekend_stay'], categories=column_values_dict['weekend_stay'])

display(interactions_df.head(15))

,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True
1,2,1,WinterVacation,[2-3],Standard,[160-260],[3-4],True
2,3,2,WinterVacation,[2-3],Standard,[160-260],[2-2],False
3,4,3,WinterVacation,[4-7],Standard,[0-160],[3-4],True
4,5,4,WinterVacation,[4-7],Standard,[0-160],[2-2],True
5,6,5,Easter,[4-7],Standard,[160-260],[5-inf],True
6,7,6,OffSeason,[2-3],Standard,[160-260],[5-inf],True
7,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True
8,9,8,HighSeason,[2-3],Standard,[0-160],[1-1],True
9,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True


# Define user features based on reservations

The content-based recommenders will be forecasting the probability of interaction between user and item based on user features vector and item features vector:

<center>
$$
    r_{u, i} = f(user\_features, item\_features)
$$
</center>

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Design numerical user features based on user reservations. Code the following method which for a given interactions DataFrame (it will be used in the fit method of the recommender) returns a DataFrame with user_id and user features as well as a list with names of user features (this will be important to select the right columns for an ML algorithm). Remember to name the columns differently than item features which you will create in the next task. Validate your features on users with several interactions (sample user ids are already given below).

Ideas for user features:
- Find the vector of most popular feature values from all user reservations and encode every feature with one-hot encoding.
- For every reservation feature calculate the probability distribution of its values among all user's reservations.
- For numerical buckets (length_of_stay, room_segment, n_people) you can calculate the average value for every user from their reservations (you will have to map the buckets back to numerical values before averaging them).

Remember that you will have to select the best features (with the highest explanatory power). Using all above features at once would make the number of variables too large for this dataset and would also introduce too much correlations between features.

You can also prepare several versions of the prepare_users_df method and test which works best in your recommender.

In [38]:
#import time

def create_users_df(interactions_df):
    df = interactions_df.copy()
    df.drop('item_id', inplace=True, axis=1)
    df = df.add_prefix('user_')
    df.rename(columns = {
        'user_user_id': 'user_id'
    }, inplace=True)
    
    return list(df.columns[1:]), df


def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return res, list(dummies.columns)
    


def get_most_frequent(df):
    most_frequent = pd.Series.mode(df)
    #print(most_frequent)
    return most_frequent.values[0] if not most_frequent.empty else np.nan


def get_most_common(srs):
    x = list(srs)
    my_counter = Counter(x)
    return my_counter.most_common(1)[0][0]


def print_most_common_in_df(df, features):
    #display(df)
    #display(features)
    for feature in features:
        print(feature)
        print(df[feature].value_counts().idxmax())
    
    """
    avg_user = {
        "user_term": "OffSeason",
        "user_length_of_stay_bucket": "[2-3]",
        "user_rate_plan": "Nonref",
        "user_room_segment": "[0-160]",
        "user_n_people_bucket": "[3-4]",
        "user_weekend_stay": "True"
    }
    """


def prepare_users_df(interactions_df):
    #start = time.time()
    
    # get user_features and prepare appropriate columns for users_df
    user_feature_titles, users_df = create_users_df(interactions_df)
    #print_most_common_in_df(users_df, user_feature_titles)
    # display(user_features)
    
    # get most frequent values for each feature
    users_df = users_df.groupby('user_id').agg(get_most_common).reset_index()
    
    # one-hot encode all features using dummies
    user_features = []
    for feature in user_feature_titles:
        users_df, feature_columns = encode_and_bind(users_df, feature)
        user_features.extend(feature_columns)
    
    #end = time.time()
    #print(end - start)

    return users_df, user_features
    

users_df, user_features = prepare_users_df(interactions_df)

print(user_features)

display(users_df.loc[users_df['user_id'].isin([706, 1736, 7779, 96, 1, 50, 115])].head(15))

['user_term_Christmas', 'user_term_Easter', 'user_term_HighSeason', 'user_term_LowSeason', 'user_term_MayLongWeekend', 'user_term_NewYear', 'user_term_OffSeason', 'user_term_WinterVacation', 'user_length_of_stay_bucket_[0-1]', 'user_length_of_stay_bucket_[2-3]', 'user_length_of_stay_bucket_[4-7]', 'user_length_of_stay_bucket_[8-inf]', 'user_rate_plan_Nonref', 'user_rate_plan_Standard', 'user_room_segment_[0-160]', 'user_room_segment_[160-260]', 'user_room_segment_[260-360]', 'user_room_segment_[360-500]', 'user_room_segment_[500-900]', 'user_n_people_bucket_[1-1]', 'user_n_people_bucket_[2-2]', 'user_n_people_bucket_[3-4]', 'user_n_people_bucket_[5-inf]', 'user_weekend_stay_False', 'user_weekend_stay_True']


,user_id,user_term_Christmas,user_term_Easter,user_term_HighSeason,user_term_LowSeason,user_term_MayLongWeekend,user_term_NewYear,user_term_OffSeason,user_term_WinterVacation,user_length_of_stay_bucket_[0-1],user_length_of_stay_bucket_[2-3],user_length_of_stay_bucket_[4-7],user_length_of_stay_bucket_[8-inf],user_rate_plan_Nonref,user_rate_plan_Standard,user_room_segment_[0-160],user_room_segment_[160-260],user_room_segment_[260-360],user_room_segment_[360-500],user_room_segment_[500-900],user_n_people_bucket_[1-1],user_n_people_bucket_[2-2],user_n_people_bucket_[3-4],user_n_people_bucket_[5-inf],user_weekend_stay_False,user_weekend_stay_True
0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1
40,50,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1
84,96,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1
102,115,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0
371,706,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1
1383,1736,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0
7301,7779,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,1


In [39]:
## TESTING STUFF ##
#display(interactions_df)
display(interactions_df.loc[interactions_df['user_id'] == 1346])
display(interactions_df.loc[interactions_df['user_id'] == 8695])
display(interactions_df.loc[interactions_df['user_id'] == 5167])
display(interactions_df.loc[(interactions_df['user_id'] == 1287) & (interactions_df['item_id'] == 162)])



,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
966,1346,260,LowSeason,[2-3],Standard,[360-500],[5-inf],False


,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
8509,8695,167,LowSeason,[4-7],Nonref,[160-260],[3-4],True


,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
14912,5167,622,WinterVacation,[2-3],Nonref,[160-260],[5-inf],True


,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
4094,1287,162,OffSeason,[4-7],Nonref,[160-260],[3-4],False


# Prepare numerical item features

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the prepare_items_df method which will be used in the recommender fit and recommend methods to map items to numerical features. This method should take the interactions_df DataFrame as input and return a DataFrame containing one record per item_id with item_id column and numerical item feature columns.

You can try turning all item features into one-hot representations. You can use the get_dummies method from pandas. It will return the same columns on any dataset of interactions because the categorical variables with all possible values have been defined in the second cell in this notebook.

You are welcome to design your own numerical item features, for instance based on numerical min and max values in buckets used as features.

In [40]:
def create_items_df(interactions_df):
    df = interactions_df.copy()
    
    if 'user_id' in df.columns:
        df.drop(['user_id'], inplace=True, axis=1)
        
    df = df.add_prefix('item_')
    df.rename(columns = {
        'item_item_id': 'item_id'
    }, inplace=True)
    
    return list(df.columns[1:]), df

def prepare_items_df(interactions_df):
    # get item_features and prepare appropriate columns for items_df
    item_features_titles, items_df = create_items_df(interactions_df)
    
    # get most frequent values for each feature
    items_df = items_df.groupby('item_id').agg(get_most_common).reset_index()
    
    # one-hot encode all features using dummies
    item_features = []
    for feature in item_features_titles:
        items_df, feature_columns = encode_and_bind(items_df, feature)
        item_features.extend(feature_columns)
    
    #display(items_df)
    return items_df, item_features


items_df, item_features = prepare_items_df(interactions_df)

print(item_features)

display(items_df.loc[items_df['item_id'].isin([0, 1, 2, 3, 4, 5, 6])].head(15))

['item_term_Christmas', 'item_term_Easter', 'item_term_HighSeason', 'item_term_LowSeason', 'item_term_MayLongWeekend', 'item_term_NewYear', 'item_term_OffSeason', 'item_term_WinterVacation', 'item_length_of_stay_bucket_[0-1]', 'item_length_of_stay_bucket_[2-3]', 'item_length_of_stay_bucket_[4-7]', 'item_length_of_stay_bucket_[8-inf]', 'item_rate_plan_Nonref', 'item_rate_plan_Standard', 'item_room_segment_[0-160]', 'item_room_segment_[160-260]', 'item_room_segment_[260-360]', 'item_room_segment_[360-500]', 'item_room_segment_[500-900]', 'item_n_people_bucket_[1-1]', 'item_n_people_bucket_[2-2]', 'item_n_people_bucket_[3-4]', 'item_n_people_bucket_[5-inf]', 'item_weekend_stay_False', 'item_weekend_stay_True']


,item_id,item_term_Christmas,item_term_Easter,item_term_HighSeason,item_term_LowSeason,item_term_MayLongWeekend,item_term_NewYear,item_term_OffSeason,item_term_WinterVacation,item_length_of_stay_bucket_[0-1],item_length_of_stay_bucket_[2-3],item_length_of_stay_bucket_[4-7],item_length_of_stay_bucket_[8-inf],item_rate_plan_Nonref,item_rate_plan_Standard,item_room_segment_[0-160],item_room_segment_[160-260],item_room_segment_[260-360],item_room_segment_[360-500],item_room_segment_[500-900],item_n_people_bucket_[1-1],item_n_people_bucket_[2-2],item_n_people_bucket_[3-4],item_n_people_bucket_[5-inf],item_weekend_stay_False,item_weekend_stay_True
0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1
1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1
2,2,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0
3,3,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1
4,4,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1
5,5,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,1
6,6,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1


# Content-based recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the content-based recommender. User features should be calculated within the fit method based on available training data and should be saved in the object as self.users_df for later use in the recommend method. Item features should be calculated both in the fit method (from interactions_df) and in the recommend method (from items_df - the items to be evaluated).

In the fit method you have to randomly generate non-existing interactions and add them to the training data for the regressor. You should add the target variable to interactions - equal to 1 for real ("positive") interactions and equal to 0 for those newly added "negative" interactions. Generate several negative interactions per every positive interaction (n_neg_per_pos). Treat the proportion as a tunable parameter of the model.

Remember to keep control over randomness - in the init method add seed as a parameter and initialize the random seed generator with that seed:

```python
self.seed = seed
self.rng = np.random.RandomState(seed=seed)
```

Below the base content-based recommender class there are several classes which inherit from the base class and use different ML models:
  - LinearRegressionCBUIRecommender - based on linear regression,
  - SVRCBUIRecommender - based on Support Vector Regressor (if you want to test it, sample the data in the fit method, as the training can take many hours on the entire dataset of interactions),
  - RandomForestCBUIRecommender - based on Random Forest,
  - XGBoostCBUIRecommender - based on XGBoost.
  
There is no need to change anything in those inheriting classes, although you can experiment with other tunable parameters of the underlying models.

You are encouraged to experiment with:
  - Other numerical user and item features (but always train and evaluate the model on buckets defined in the first notebook).
  - Other ML models, e.g. Huber regression, Lasso regression, Ridge regression, LARS regression, Linear SVR, Decision Tree, Naive Bayes, LightGBM, Neural Networks or any model of your choice.

In [41]:
def rand_negative_interactions(interactions_df, rng, negative_interactions_size):
    negative_interactions = []
    
    user_ids = interactions_df['user_id'].unique()
    item_ids = interactions_df['item_id'].unique()

    random_users = rng.choice(user_ids, negative_interactions_size)
    random_items = rng.choice(item_ids, negative_interactions_size)

    for interaction_counter in range(negative_interactions_size):
        negative_interactions.append(tuple((
            random_users[interaction_counter], 
            random_items[interaction_counter], 
            0
        )))
        
    return negative_interactions


def rand_correct_negative_interactions(interactions_df, rng, negative_interactions_size):
    negative_interactions = []
    
    max_user = interactions_df['user_id'].max()
    max_item = interactions_df['item_id'].max()

    # print(max_user, max_item)
    
    positive_interactions = interactions_df[['user_id', 'item_id']].to_numpy()
    
    while (len(negative_interactions) < negative_interactions_size):
        user_id = rng.choice(max_user + 1)
        item_id = rng.choice(max_item + 1)
        
        negative_interaction = tuple((user_id, item_id))
        
        if not (negative_interaction == positive_interactions).all(axis=1).any():
            negative_interactions.append(tuple((user_id, item_id, 0)))

            
    return negative_interactions

In [42]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from recommenders.recommender import Recommender


class ContentBasedUserItemRecommender(Recommender):
    """
    Linear recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5):
        """
        Initialize base recommender params and variables.
        """
        self.model = LinearRegression()
        self.n_neg_per_pos = n_neg_per_pos
        
        self.recommender_df = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        self.users_df = None
        self.user_features = None
        
        self.seed = seed
        self.rng = np.random.RandomState(seed=seed)
    
    def fit(self, interactions_df, users_df, items_df):
        #start = time.time()
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        interactions_df = interactions_df.copy()
        
        # Prepare users_df and items_df
        users_df, user_features = prepare_users_df(interactions_df)
        
        self.users_df = users_df
        self.user_features = user_features
        
        items_df, item_features = prepare_items_df(interactions_df)
        items_df = items_df.loc[:, ['item_id'] + item_features]
        
        # Generate negative interactions
        interactions_df = interactions_df.loc[:, ['user_id', 'item_id']]
        
        interactions_df.loc[:, 'interacted'] = 1
              
        negative_interactions_size = int(len(interactions_df) * self.n_neg_per_pos)
        negative_interactions = rand_correct_negative_interactions(interactions_df, self.rng, negative_interactions_size)
        
        interactions_df = pd.concat(
            [interactions_df, pd.DataFrame(negative_interactions, columns=['user_id', 'item_id', 'interacted'])])
        
        # Get the input data for the model
        interactions_df = pd.merge(interactions_df, users_df, on=['user_id'])
        interactions_df = pd.merge(interactions_df, items_df, on=['item_id'])
        
        x = interactions_df.loc[:, user_features + item_features].values
        y = interactions_df['interacted'].values
        
        #end = time.time()
        #print(end - start)
        self.model.fit(x, y)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        # Clean previous recommendations (iloc could be used alternatively)
        self.recommender_df = self.recommender_df[:0]
        
        # prepare avg_user
        avg_user = {
            'user_term_Christmas': 0, 'user_term_Easter': 0, 'user_term_HighSeason': 0, 'user_term_LowSeason': 0, 
            'user_term_MayLongWeekend': 0, 'user_term_NewYear': 0, 'user_term_OffSeason': 1, 
            'user_term_WinterVacation': 0, 
            'user_length_of_stay_bucket_[0-1]': 0, 'user_length_of_stay_bucket_[2-3]': 1, 
            'user_length_of_stay_bucket_[4-7]': 0, 'user_length_of_stay_bucket_[8-inf]': 0, 
            'user_rate_plan_Nonref': 1, 'user_rate_plan_Standard': 0, 
            'user_room_segment_[0-160]': 1, 'user_room_segment_[160-260]': 0, 
            'user_room_segment_[260-360]': 0, 'user_room_segment_[360-500]': 0, 
            'user_room_segment_[500-900]': 0, 
            'user_n_people_bucket_[1-1]': 0, 'user_n_people_bucket_[2-2]': 0, 
            'user_n_people_bucket_[3-4]': 1, 'user_n_people_bucket_[5-inf]': 0, 
            'user_weekend_stay_False': 0, 'user_weekend_stay_True': 1
        }

        #avg_user_df = self.users_df.loc[self.users_df['user_id'] == 96]
        #avg_user_dict = avg_user_df.to_dict()
        #avg_user_dict.pop('user_id', None)
        #print(avg_user_dict)
        
        
        # get data for existing users, fill avg_user for rest
        users_df = pd.merge(
            users_df, 
            self.users_df, 
            on='user_id', 
            how='left'
        ).fillna(value=avg_user) 
        
        #display(users_df)
        
        # prepare items_df
        items_df, item_features = prepare_items_df(items_df)
        
        # Score the items
    
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            
            # Create a Carthesian product of users from users_df and items from items_df
            user_df = pd.DataFrame(user).transpose()
            # display(user_df)
            
            user_item_cartesian_product = user_df.merge(items_df, how="cross")
            user_item_cartesian_product.drop(['user_id', 'item_id'], axis=1, inplace=True)
            #display(user_item_cartesian_product)
            
            # Use self.model.predict method to calculate scores for all records in the just created DataFrame
            # of users and items
            # display(user_item_cartesian_product.values)
            
            #display(np.any(np.isnan(user_item_cartesian_product.values)))
            #display(np.all(np.isfinite(user_item_cartesian_product.values)))
            
            scores = self.model.predict(user_item_cartesian_product.values)
            
            # Obtain item ids with the highest score and save those ids under the chosen_ids variable
            # Do not exclude already booked items.
            chosen_ids = np.argsort(-scores)[:n_recommendations]
            #display(chosen_ids)
            
            recommendations = []
            for item_id in chosen_ids:
                recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': item_id,
                        'score': scores[item_id]
                    }
                )
            
            user_recommendations = pd.DataFrame(recommendations)

            self.recommender_df = pd.concat([self.recommender_df, user_recommendations])

        return self.recommender_df
    
    
class LinearRegressionCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Linear regression recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        self.model = LinearRegression()
        
        
class SVRCBUIRecommender(ContentBasedUserItemRecommender):
    """
    SVR recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'kernel' in model_params:
            self.kernel = model_params['kernel']
        else:
            self.kernel = 'rbf'
        if 'C' in model_params:
            self.C = model_params['C']
        else:
            self.C = 1.0
        if 'epsilon' in model_params:
            self.epsilon = model_params['epsilon']
        else:
            self.epsilon = 0.1
        self.model = SVR(kernel=self.kernel, C=self.C, epsilon=self.epsilon)
        
    
class RandomForestCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Random forest recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        self.model = RandomForestRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split)
    
    
class XGBoostCBUIRecommender(ContentBasedUserItemRecommender):
    """
    XGBoost recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        if 'learning_rate' in model_params:
            self.learning_rate = model_params['learning_rate']
        else:
            self.learning_rate = 30
        self.model = GradientBoostingRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split,
            learning_rate=self.learning_rate)    

# Quick test of the recommender

In [43]:
items_df = interactions_df.loc[:, ['item_id'] + base_item_features].drop_duplicates()

In [44]:
# Fit method
cb_user_item_recommender = LinearRegressionCBUIRecommender()
cb_user_item_recommender.fit(interactions_df, None, None)

In [45]:
# Recommender method

recommendations = cb_user_item_recommender.recommend(pd.DataFrame([[1], [2], [3], [4], [5], [9]], columns=['user_id']), interactions_df, 10)

recommendations = pd.merge(recommendations, items_df, on='item_id', how='left')
display(recommendations)

,user_id,item_id,score,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,56,0.553894,OffSeason,[2-3],Nonref,[0-160],[3-4],True
1,1,22,0.544495,OffSeason,[2-3],Standard,[0-160],[3-4],True
2,1,60,0.521423,OffSeason,[2-3],Nonref,[0-160],[2-2],True
3,1,33,0.511902,OffSeason,[2-3],Standard,[0-160],[2-2],True
4,1,93,0.510071,OffSeason,[2-3],Nonref,[160-260],[3-4],True
5,1,24,0.500793,OffSeason,[2-3],Standard,[160-260],[3-4],True
6,1,151,0.477722,OffSeason,[2-3],Nonref,[160-260],[2-2],True
7,1,129,0.475159,OffSeason,[2-3],Nonref,[0-160],[3-4],False
8,1,1019,0.474792,HighSeason,[2-3],Nonref,[0-160],[3-4],True
9,1,35,0.468201,OffSeason,[2-3],Standard,[160-260],[2-2],True


# Tuning method

In [46]:
from evaluation_and_testing.testing import evaluate_train_test_split_implicit

seed = 6789

In [47]:
from hyperopt import hp, fmin, tpe, Trials
import traceback

def tune_recommender(recommender_class, interactions_df, items_df, 
                     param_space, max_evals=1, show_progressbar=True, seed=6789):
    # Split into train_validation and test sets

    shuffle = np.arange(len(interactions_df))
    rng = np.random.RandomState(seed=seed)
    rng.shuffle(shuffle)
    shuffle = list(shuffle)

    train_test_split = 0.8
    split_index = int(len(interactions_df) * train_test_split)

    train_validation = interactions_df.iloc[shuffle[:split_index]]
    test = interactions_df.iloc[shuffle[split_index:]]

    # Tune

    def loss(tuned_params):
        recommender = recommender_class(seed=seed, **tuned_params)
        hr1, hr3, hr5, hr10, ndcg1, ndcg3, ndcg5, ndcg10 = evaluate_train_test_split_implicit(
            recommender, train_validation, items_df, seed=seed)
        return -hr10

    n_tries = 1
    succeded = False
    try_id = 0
    while not succeded and try_id < n_tries:
        try:
            trials = Trials()
            best_param_set = fmin(loss, space=param_space, algo=tpe.suggest, 
                                  max_evals=max_evals, show_progressbar=show_progressbar, trials=trials, verbose=True)
            succeded = True
        except:
            traceback.print_exc()
            try_id += 1
            
    if not succeded:
        return None
        
    # Validate
    
    recommender = recommender_class(seed=seed, **best_param_set)

    results = [[recommender_class.__name__] + list(evaluate_train_test_split_implicit(
        recommender, {'train': train_validation, 'test': test}, items_df, seed=seed))]

    results = pd.DataFrame(results, 
                           columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

    display(results)
    
    return best_param_set

## Tuning of the recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Tune your models using the code below. You only need to put the class name of your recommender and choose an appropriate parameter space.

In [18]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1)
}

best_param_set = tune_recommender(LinearRegressionCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=10, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

100%|██████████| 10/10 [3:57:34<00:00, 1425.47s/trial, best loss: -0.177316969953449] 


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.03632,0.099796,0.143245,0.186354,0.03632,0.071347,0.088855,0.102921


Best parameters:
{'n_neg_per_pos': 6.0}


In [ ]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'C': hp.loguniform('C', np.log(0.01), np.log(100.0))
}

best_param_set = tune_recommender(SVRCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

  6%|▌         | 6/100 [5:51:42<97:31:07, 3734.76s/trial, best loss: -0.20016927634363096] 

In [85]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 8, 1),
    'n_estimators': hp.quniform('n_estimators', 30, 250, 1),
    'max_depth': hp.quniform('max_depth', 2, 8, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 25, 1)
}

best_param_set = tune_recommender(RandomForestCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

2.879838228225708                                      
3.415518045425415                                      
2.8268725872039795                                                                      
4.307072639465332                                                                       
2.7973594665527344                                                                      
4.287855386734009                                                                       
2.8360981941223145                                                                      
3.386230707168579                                                                       
2.7907097339630127                                                                        
3.3215487003326416                                                                        
2.8379931449890137                                                                        
3.9146885871887207                                                               

3.8108160495758057                                                                        
2.823961019515991                                                                         
3.3652503490448                                                                           
3.9730770587921143                                                                        
4.9378368854522705                                                                        
6.817636966705322                                                                         
7.926626443862915                                                                         
3.369194984436035                                                                         
4.126676797866821                                                                         
5.3850038051605225                                                                        
6.972212314605713                                                                         

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RandomForestCBUIRecommender,0.025119,0.069586,0.081127,0.141208,0.025119,0.048152,0.052736,0.072517


Best parameters:
{'max_depth': 2.0, 'min_samples_split': 6.0, 'n_estimators': 169.0, 'n_neg_per_pos': 2.0}


In [24]:
# This tuning may take around 12 hours

param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 10, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1))
}

best_param_set = tune_recommender(XGBoostCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

 18%|█▊        | 18/100 [5:41:55<25:57:38, 1139.74s/trial, best loss: -0.18662716885315278]
Best parameters:
None


Traceback (most recent call last):
  File "/home/mateusz/anaconda3/envs/rs-class-env/lib/python3.8/site-packages/pandas/core/generic.py", line 5477, in __setattr__
    object.__getattribute__(self, name)
  File "/home/mateusz/anaconda3/envs/rs-class-env/lib/python3.8/site-packages/pandas/core/series.py", line 536, in name
    return self._name
  File "/home/mateusz/anaconda3/envs/rs-class-env/lib/python3.8/site-packages/pandas/core/generic.py", line 5461, in __getattr__
    return object.__getattribute__(self, name)
AttributeError: 'Series' object has no attribute '_name'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-12-0fde002c7dd7>", line 33, in tune_recommender
    best_param_set = fmin(loss, space=param_space, algo=tpe.suggest,
  File "/home/mateusz/anaconda3/envs/rs-class-env/lib/python3.8/site-packages/hyperopt/fmin.py", line 540, in fmin
    return trials.fmin(
  File "/home/mateusz/anaconda3/envs/

# Final evaluation

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Run the final evaluation of your recommender and present its results against the Amazon recommender's results. You can present results for several of your recommenders. You just need to give the class name of your recommender and its tuned parameters below. If you present results for several recommenders, you should add a separate cell for each recommender and change the names of the DataFrames containing results.

In [19]:
cb_user_item_recommender_2 = RandomForestCBUIRecommender(
    **{
        'n_neg_per_pos': 10
    })  # Initialize your recommender here with the best params from tuning

# Give the name of your recommender in the line below
random_forest_1_cbui_tts_results = [['RandomForestCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender_2, interactions_df, items_df))]

random_forest_1_cbui_tts_results = pd.DataFrame(
    random_forest_1_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(random_forest_1_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RandomForestCBUIRecommender,0.046164,0.10964,0.144263,0.190767,0.046164,0.081369,0.095938,0.110839


In [20]:
cb_user_item_recommender_3 = RandomForestCBUIRecommender(
    **{
        'max_depth': 2.0, 
        'min_samples_split': 6.0, 
        'n_estimators': 169.0, 
        'n_neg_per_pos': 2.0
    })  # Initialize your recommender here with the best params from tuning

# Give the name of your recommender in the line below
random_forest_2_cbui_tts_results = [['RandomForestCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender_3, interactions_df, items_df))]

random_forest_2_cbui_tts_results = pd.DataFrame(
    random_forest_2_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(random_forest_2_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RandomForestCBUIRecommender,0.002376,0.004413,0.005092,0.010183,0.002376,0.003572,0.00385,0.005494


In [28]:
cb_user_item_recommender_5 = XGBoostCBUIRecommender(
    **{
        'n_neg_per_pos': 7.0
    })  # Initialize your recommender here with the best params from tuning

# Give the name of your recommender in the line below
XG_Boost_4_cbui_tts_results = [['XGBoostCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender_5, interactions_df, items_df))]

XG_Boost_4_cbui_tts_results = pd.DataFrame(
    XG_Boost_4_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(XG_Boost_4_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,XGBoostCBUIRecommender,0.0,0.002037,0.005431,0.010862,0.0,0.001285,0.002747,0.004445


In [48]:
cb_user_item_recommender = LinearRegressionCBUIRecommender(
    **{
        'n_neg_per_pos': 6
    })  # Initialize your recommender here with the best params from tuning

# Give the name of your recommender in the line below
linear_cbui_tts_results = [['LinearRegressionCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

linear_cbui_tts_results = pd.DataFrame(
    linear_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(linear_cbui_tts_results)


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.03632,0.099796,0.143245,0.186354,0.03632,0.071347,0.088855,0.102921


In [22]:
from recommenders.amazon_recommender import AmazonRecommender

amazon_recommender = AmazonRecommender()

amazon_tts_results = [['AmazonRecommender'] + list(evaluate_train_test_split_implicit(
    amazon_recommender, interactions_df, items_df))]

amazon_tts_results = pd.DataFrame(
    amazon_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(amazon_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,AmazonRecommender,0.035981,0.09165,0.132043,0.185336,0.035981,0.068082,0.084824,0.102078


In [30]:
tts_results = pd.concat([
    linear_cbui_tts_results, 
    random_forest_2_cbui_tts_results,
    random_forest_1_cbui_tts_results,
    XG_Boost_4_cbui_tts_results,
    amazon_tts_results
]).reset_index(drop=True)

display(tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.036320,0.099796,0.143245,0.186354,0.036320,0.071347,0.088855,0.102921
1,RandomForestCBUIRecommender,0.002376,0.004413,0.005092,0.010183,0.002376,0.003572,0.003850,0.005494
2,RandomForestCBUIRecommender,0.046164,0.109640,0.144263,0.190767,0.046164,0.081369,0.095938,0.110839
3,XGBoostCBUIRecommender,0.000000,0.002037,0.005431,0.010862,0.000000,0.001285,0.002747,0.004445
4,AmazonRecommender,0.035981,0.091650,0.132043,0.185336,0.035981,0.068082,0.084824,0.102078
